# CIFAR Conv Net

И так, в этом ноутбуке Вы сделаете превую в своей жизни сверточную сеть! На сложном датасете. Cкачайте его кстати, 

!mkdir cifar10
!curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz -C cifar10

In [1]:
import numpy as np

In [2]:
from cifar import load_CIFAR10

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [3]:
#norm it
X_train /= np.float32(255.)
X_test /= np.float32(255.)
mean = X_train.mean(0)
X_train -= mean
X_test -= mean
# add flipped data
X_train_flip = X_train[:,:,:,::-1]
y_train_flip = y_train
X_train = np.concatenate((X_train,X_train_flip),axis=0)
y_train = np.concatenate((y_train,y_train_flip),axis=0)

<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Чем отличаются современные сверточные сети от сетей 5 летней давности?

<Ответ>

**Вопрос 2**: Какие неприятности могут возникнуть во время обучения современных нейросетей?

<Ответ>


**Вопрос 3**: У вас есть очень маленький датасет из 100 картинок, классификация, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить? что делать если первый вариант  решения не заработает?

<Ответ>

**Вопрос 4**: Как сделать стайл трансфер для музыки? oO

<Ответ>

In [4]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

input_X = T.tensor4("X")

target_y = T.vector("target Y integer",dtype='int32')

Using cuDNN version 5110 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:04.0)


### Соберите нейронку: 
- Many times x (Conv+Pool)
- Many small convolutions like 3x3
- Batch Norm 
- Residual Connection
- Data Augmentation 
- Learning rate Schedule 
- ...

### Для вдохновения 
- http://torch.ch/blog/2015/07/30/cifar.html
- https://github.com/szagoruyko/wide-residual-networks 

### Самое интересное
- Для сдачи задания нужно набрать на точность тесте > **92.5**% (это займет много времени, торопитесь :) )
- Для получения бонусных баллов > **95.0**%
- Будет очень хорошо если вы придумаете свою архитектуру или сможете обучить что-то из вышеперечисленного :)
- А для обучения всего этого добра вам будет куда удобнее использовать GPU на Amazon 
    - Инструкция https://github.com/persiyanov/ml-mipt/tree/master/amazon-howto 
    - Вам помогут tmux, CuDNN, ssh tunnel, nvidia-smi, ... 
    - Have fun :)

In [5]:
from resent_git import build_cnn

In [7]:
net = build_cnn(input_var=input_X, n=9)

with np.load('pre_trained/cifar_model_n9.npz') as f:
     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values(net, param_values)

In [8]:
y_predicted = lasagne.layers.get_output(net)
all_weights = lasagne.layers.get_all_params(net, trainable=True)

In [9]:
print len(all_weights)

167


In [10]:
LR_SCHEDULE = {
    0: 0.1,
    40: 0.01,
    60: 0.001
}

In [11]:
l2_penalty = lasagne.regularization.regularize_layer_params(lasagne.layers.get_all_layers(net), 
                                                            lasagne.regularization.l2) * 0.0001
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean() + l2_penalty
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [12]:
l_r = theano.shared(lasagne.utils.floatX(LR_SCHEDULE[0]))
train_fun = theano.function([input_X,target_y],[loss, accuracy], 
                            updates=lasagne.updates.momentum(loss, all_weights, 
                                                        learning_rate=l_r),
                            allow_input_downcast=True)
accuracy_fun = theano.function([input_X,target_y],accuracy, allow_input_downcast=True)

### Вот и всё, пошли её учить

In [13]:
from numpy import random

def iterate_minibatches(inputs, targets, batchsize, shuffle=False, augment=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        if augment:
            padded = np.pad(inputs[excerpt],((0,0),(0,0),(4,4),(4,4)),mode='constant')
            random_cropped = np.zeros(inputs[excerpt].shape, dtype=np.float32)
            crops = np.random.randint(0,high=9,size=(batchsize,2))
            for r in range(batchsize):
                random_cropped[r,:,:,:] = padded[r,:,crops[r,0]:(crops[r,0]+32),crops[r,1]:(crops[r,1]+32)]
            inp_exc = random_cropped
        else:
            inp_exc = inputs[excerpt]

        yield inp_exc, targets[excerpt]

# Процесс обучения

In [14]:
import time

batch_size = 128 #размер мини-батча

num_epochs = 82 #количество проходов по данным

with open('log-try-2.txt', 'w') as f:
    f.write('LR_SCHEDULE: {}\n'.format(LR_SCHEDULE.__repr__()))
for epoch in range(num_epochs):

    if epoch in LR_SCHEDULE:
            l_r.set_value(LR_SCHEDULE[epoch])
            
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    # In each epoch, we do a full pass over the training data:
    for batch in iterate_minibatches(X_train, y_train,batch_size,shuffle=True, augment=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))
    with open('log-try-2.txt', 'a+') as f:
        f.write("Epoch {} of {} took {:.3f}s\n".format(epoch + 1, num_epochs, time.time() - start_time))
        f.write("  training loss (in-iteration):\t\t{:.6f}\n".format(train_err / train_batches))
        f.write("  train accuracy:\t\t{:.2f} %\n".format(train_acc / train_batches * 100))
        f.write("  validation accuracy:\t\t{:.2f} %\n".format(val_acc / val_batches * 100))
        

Epoch 1 of 82 took 250.384s
  training loss (in-iteration):		2.493403
  train accuracy:		31.69 %
  validation accuracy:		48.98 %
Epoch 2 of 82 took 250.259s
  training loss (in-iteration):		1.730353
  train accuracy:		54.53 %
  validation accuracy:		64.74 %
Epoch 3 of 82 took 250.426s
  training loss (in-iteration):		1.305190
  train accuracy:		68.36 %
  validation accuracy:		72.74 %
Epoch 4 of 82 took 249.958s
  training loss (in-iteration):		1.087197
  train accuracy:		74.46 %
  validation accuracy:		76.46 %
Epoch 5 of 82 took 250.206s
  training loss (in-iteration):		0.963387
  train accuracy:		77.83 %
  validation accuracy:		78.76 %
Epoch 6 of 82 took 250.463s
  training loss (in-iteration):		0.880794
  train accuracy:		79.95 %
  validation accuracy:		80.68 %
Epoch 7 of 82 took 250.266s
  training loss (in-iteration):		0.812106
  train accuracy:		81.83 %
  validation accuracy:		82.01 %
Epoch 8 of 82 took 249.856s
  training loss (in-iteration):		0.769688
  train accuracy:		83.25 %


Epoch 65 of 82 took 249.629s
  training loss (in-iteration):		0.204260
  train accuracy:		99.80 %
  validation accuracy:		92.22 %
Epoch 66 of 82 took 249.752s
  training loss (in-iteration):		0.203390
  train accuracy:		99.81 %
  validation accuracy:		92.20 %
Epoch 67 of 82 took 249.565s
  training loss (in-iteration):		0.202578
  train accuracy:		99.83 %
  validation accuracy:		92.20 %
Epoch 68 of 82 took 249.826s
  training loss (in-iteration):		0.201781
  train accuracy:		99.84 %
  validation accuracy:		92.24 %
Epoch 69 of 82 took 249.824s
  training loss (in-iteration):		0.201148
  train accuracy:		99.83 %
  validation accuracy:		92.24 %
Epoch 70 of 82 took 249.729s
  training loss (in-iteration):		0.200518
  train accuracy:		99.84 %
  validation accuracy:		92.16 %
Epoch 71 of 82 took 249.653s
  training loss (in-iteration):		0.199521
  train accuracy:		99.85 %
  validation accuracy:		92.25 %
Epoch 72 of 82 took 249.821s
  training loss (in-iteration):		0.199035
  train accuracy:		

In [15]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 92.5:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"
    

with open('log-try-2', 'a+') as f:
    f.write("Final results:\n")
    f.write("  test accuracy:\t\t{:.2f} %\n".format(
        test_acc / test_batches * 100))

    if test_acc / test_batches * 100 > 92.5:
        f.write("Achievement unlocked: колдун 80 уровня\n")
    else:
        f.write("Нужно больше магии!\n")

Final results:
  test accuracy:		92.42 %
Нужно больше магии!


In [18]:
test_prediction = lasagne.layers.get_output(net, deterministic=True)
test_accuracy = lasagne.objectives.categorical_accuracy(test_prediction, target_y).mean()
test_accuracy_fun = theano.function([input_X,target_y],test_accuracy, allow_input_downcast=True)

In [19]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = test_accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 92.5:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"
    

with open('log-try-2', 'a+') as f:
    f.write("Final results:\n")
    f.write("  test accuracy:\t\t{:.2f} %\n".format(
        test_acc / test_batches * 100))

    if test_acc / test_batches * 100 > 92.5:
        f.write("Achievement unlocked: колдун 80 уровня\n")
    else:
        f.write("Нужно больше магии!\n")

Final results:
  test accuracy:		92.37 %
Нужно больше магии!


In [20]:
adam_train_fun = theano.function([input_X,target_y],[loss, accuracy], 
                            updates=lasagne.updates.adam(loss, all_weights, 
                                                        learning_rate=0.000001),
                            allow_input_downcast=True)

In [21]:
train_err = 0
train_acc = 0
train_batches = 0
# In each epoch, we do a full pass over the training data:
for batch in iterate_minibatches(X_train, y_train,batch_size,shuffle=True, augment=True):
    inputs, targets = batch
    train_err_batch, train_acc_batch= adam_train_fun(inputs, targets)
    train_err += train_err_batch
    train_acc += train_acc_batch
    train_batches += 1
print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))

  training loss (in-iteration):		0.192249
  train accuracy:		99.87 %


In [22]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = test_accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 92.5:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"
    

with open('log-try-2', 'a+') as f:
    f.write("Final results:\n")
    f.write("  test accuracy:\t\t{:.2f} %\n".format(
        test_acc / test_batches * 100))

    if test_acc / test_batches * 100 > 92.5:
        f.write("Achievement unlocked: колдун 80 уровня\n")
    else:
        f.write("Нужно больше магии!\n")

Final results:
  test accuracy:		93.04 %
Achievement unlocked: колдун 80 уровня


## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 